[View in Colaboratory](https://colab.research.google.com/github/Hoiy/kaggle-home-credit-default-risk/blob/master/model_nn.ipynb)

In [1]:
import pandas as pd
import numpy as np
import dotenv
from scipy import sparse
import pickle
import os

dotenv.load_dotenv('.env')

True

In [2]:
!mkdir prep
!gsutil rsync gs://{os.environ['GCP_BUCKET']}/prep prep

mkdir: cannot create directory ‘prep’: File exists
Building synchronization state...
Starting synchronization...


In [3]:
train = pd.read_parquet('./prep/application_train.snappy.parquet')
train.shape

test = pd.read_parquet('./prep/application_test.snappy.parquet')
test.shape

(48744, 1561)

In [4]:
y_train = train.TARGET
X_train = train.drop(columns=['TARGET'])
X_test = test

del train
del test
import gc
gc.collect()

# def get_category_cols(df):
#   cols = []
#   for idx, dtype in df.dtypes.iteritems():
#     if dtype in ['object', 'int64'] and (len(df[idx].unique()) / len(df[idx].dropna()) <= 0.1):
#       cols.append(idx)

#   return cols

# cat_cols = get_category_cols(X_train)

from tqdm import tqdm

cat_cols = []
cont_cols = []
for col, dtype in tqdm(X_train.dtypes.iteritems()):
  if dtype == 'object':
    temp = pd.concat([X_train[col], X_test[col]]).astype('category')
    X_train[col] = X_train[col].astype(temp.dtypes)
    X_test[col] = X_test[col].astype(temp.dtypes)
    cat_cols.append(col)
  else:
    cont_cols.append(col)

1561it [01:39, 15.67it/s]


3152

In [0]:
X_train.to_feather('X_train.feather')
y_train.to_frame().to_feather('y_train.feather')
X_test.to_feather('X_test.feather')
import pickle
with open('cols_type.pkl', 'wb') as f:
  pickle.dump([cat_cols, cont_cols], f)

In [2]:
X_train = pd.read_feather('X_train.feather')
y_train = pd.read_feather('y_train.feather')['TARGET']
import pickle
with open('cols_type.pkl', 'rb') as f:
  cat_cols, cont_cols = pickle.load(f)

/usr/local/lib/python3.6/dist-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [6]:
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Flatten, Activation, Reshape, Add, Average
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, GRU, Conv1D, Reshape, MaxPooling1D, Concatenate, Dot
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2, l1
from keras.constraints import non_neg, unit_norm
import keras.backend as K
from keras.metrics import mse
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier

DROPOUT = 0.2
# REGULARIZATION = 1e-5
# EMB_SIZE = 1

import tensorflow as tf

def roc_auc_score(y_true, y_pred):
    """ ROC AUC Score.
    Approximates the Area Under Curve score, using approximation based on
    the Wilcoxon-Mann-Whitney U statistic.
    Yan, L., Dodier, R., Mozer, M. C., & Wolniewicz, R. (2003).
    Optimizing Classifier Performance via an Approximation to the Wilcoxon-Mann-Whitney Statistic.
    Measures overall performance for a full range of threshold levels.
    Arguments:
        y_pred: `Tensor`. Predicted values.
        y_true: `Tensor` . Targets (labels), a probability distribution.
    """
    with tf.name_scope("RocAucScore"):

        pos = tf.boolean_mask(y_pred, tf.cast(y_true, tf.bool))
        neg = tf.boolean_mask(y_pred, ~tf.cast(y_true, tf.bool))

        pos = tf.expand_dims(pos, 0)
        neg = tf.expand_dims(neg, 1)

        # original paper suggests performance is robust to exact parameter choice
        gamma = 0.2
        p     = 3

        difference = tf.zeros_like(pos * neg) + pos - neg - gamma

        masked = tf.boolean_mask(difference, difference < 0.0)

        return tf.reduce_sum(tf.pow(-masked, p))
      
      
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc
  
  

def build_model():
#     zeros = Input(shape=(1, ))
#     ones = Input(shape=(1, ))
#     cat_inputs = []
#     cat_embs = []
#     for cat, s in zip(cat_cols, emb_size):
#       inp = Input(shape=(1,), name=cat)
#       emb = Embedding(train[cat].max()+1, s, embeddings_regularizer=l2(REGULARIZATION))(inp)
#       emb = Flatten()(emb)
#       emb = Activation('tanh')(emb)
      
#       cat_inputs.append(inp)
#       cat_embs.append(emb)
    
    cont_inp = Input(shape=(len(cont_cols),))      

    emb = Dropout(DROPOUT)(cont_inp)
    emb = Dense(256, activation='selu')(emb)
    emb = Dropout(DROPOUT)(emb)
    emb = Dense(256, activation='selu')(emb)
    emb = Dropout(DROPOUT)(emb)
    out = Dense(1, activation='sigmoid')(emb)
    
    model = Model(inputs=cont_inp, outputs=out)
    model.compile(
#         loss=roc_auc_score,
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['binary_crossentropy', auc])

    return model

model = build_model()
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1393)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1393)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               356864    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total para

In [0]:
# val_period = ((train['activation_date'] >= '2017-03-15') & (train['activation_date'] <= '2017-03-16')) |\
#   ((train['activation_date'] >= '2017-03-22') & (train['activation_date'] <= '2017-03-23'))
# train_period = ~val_period

# BATCH_SIZE = 4096 * 2
EPOCHS = 20000
FILE_PATH = 'nn_model.h5'

checkpoint = ModelCheckpoint(FILE_PATH, monitor='val_loss', save_best_only=True)
early = EarlyStopping(monitor="val_loss", mode="min", patience=5)

callbacks_list = [checkpoint, early] #early

In [0]:
# from sklearn.model_selection import KFold, cross_val_score


# k_fold = KFold(n_splits=5, shuffle=True)
# model = KerasRegressor(build_model, batch_size=BATCH_SIZE)
# metrics = cross_val_score(
#     model, 
#     [train[col] for col in cat_cols] + [train['price_std']], 
#     train['deal_probability'], 
#     cv=k_fold,
#     n_jobs=1,
#     fit_params={
#         'validation_split': 0.2,
#         'shuffle': True,
#         'batch_size': BATCH_SIZE,
#         'epochs': EPOCHS,
#         'callbacks': callbacks_list
#     }
# )

In [0]:
# Full training
# train_inp = [train[train_period][col] for col in cat_cols] + [train[train_period][col] for col in cont_cols]
# val_inp = [train[val_period][col] for col in cat_cols] + [train[val_period][col] for col in cont_cols]

BATCH_SIZE = 64

# model = KerasRegressor(build_model, batch_size=BATCH_SIZE)
# model.fit(train_inp, train[train_period]['deal_probability'], **{
#         'validation_data': (val_inp, train[val_period]['deal_probability']),
#         'shuffle': True,
#         'batch_size': BATCH_SIZE,
#         'epochs': EPOCHS,
#         'callbacks': callbacks_list
# })
model = KerasClassifier(build_model, batch_size=BATCH_SIZE)
model.fit(X_train[cont_cols], y_train, **{
        'validation_split': 0.2,
        'shuffle': True,
        'batch_size': BATCH_SIZE,
        'epochs': EPOCHS,
        'callbacks': callbacks_list
})

In [0]:
# from keras.models import load_model
# model = load_model(FILE_PATH, custom_objects={'rmse': rmse})

metric = model.score([train[col] for col in cat_cols] + [train[col] for col in cont_cols], train['deal_probability'])

1503424/1503424 [==============================] - 17s 11us/step


In [0]:
metric

-0.21996117967432383

In [0]:
SUBMISSION_FILE='baseline.csv'
SUBMISSION_MESSAGE='"Baseline %f"'%metric

test['deal_probability'] = model.predict(
    [test[col] for col in cat_cols] + [test[col] for col in cont_cols],
    batch_size=BATCH_SIZE
)
test[['item_id', 'deal_probability']].to_csv(SUBMISSION_FILE, index=False)

In [0]:
len(test['item_id']) == len(submission['item_id'])

True

In [0]:
!kaggle competitions submit -f '{SUBMISSION_FILE}' -m '{SUBMISSION_MESSAGE}'

Using competition: avito-demand-prediction
Successfully submitted to Avito Demand Prediction Challenge

In [0]:
!kaggle competitions submit

usage: kaggle competitions submit [-h] [-c COMPETITION] -f FILE_NAME -m
                                  MESSAGE [-q]
kaggle competitions submit: error: the following arguments are required: -f/--file, -m/--message


In [0]:
!echo "{SUBMISSION_FILE}"

baseline.csv


In [0]:
!kaggle competitions submit -m 'Baseline -0.22988' -f baseline.csv

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 11, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.6/dist-packages/kaggle/cli.py", line 48, in main
    out = args.func(**command_args)
TypeError: competition_submit_cli() got an unexpected keyword argument 'file_name'


In [0]:
!pip freeze | grep kaggle

kaggle==1.3.9


In [0]:
!kaggle --version

Kaggle API 1.3.8
